In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [3]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [4]:
import torch

In [5]:
data = pd.read_csv('LabExamCleanO10K.csv')
data.head(10)

,Unnamed: 0,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc
0,0,1000100164439,2016-05-20,Thyroid-Stimulating Hormone (TSH),2.51,mU/L
1,1,1000100164719,2016-02-22,Thyroid-Stimulating Hormone (TSH),1.37,mU/L
2,2,1000100166298,2016-01-12,Thyroid-Stimulating Hormone (TSH),3.52,mU/L
3,3,1000100164439,2016-06-17,Leukocytes in Blood (WBC),5.80,10^9/L
4,4,1000100164439,2016-05-20,Leukocytes in Blood (WBC),7.00,10^9/L
5,5,1000100164719,2013-10-11,Erythrocyte Mean Corpuscular Volume (MCV),92.00,fL
6,6,1000100164719,2013-09-22,Erythrocyte Mean Corpuscular Volume (MCV),93.00,fL
7,7,1000100164439,2013-11-06,Platelets in Blood (PLT),239.00,10^9/L
8,8,1000100164719,2013-10-11,Platelets in Blood (PLT),225.00,10^9/L
9,9,1000100164719,2013-09-22,Platelets in Blood (PLT),243.00,10^9/L


In [6]:
data = pd.read_csv('LabExamCleanO10K.csv')
alb_data = data[['Patient_ID', 'PerformedDate', 'TestResult_calc', 'Name_calc']]
alb_data = alb_data.loc[data['Name_calc'] == 'Hemoglobin A1c (HbA1c)']
alb_data = alb_data.dropna()
alb_data.head(10)

,Patient_ID,PerformedDate,TestResult_calc,Name_calc
20,1000100164439,2016-10-28,7.8,Hemoglobin A1c (HbA1c)
21,1000100165244,2016-10-14,6.8,Hemoglobin A1c (HbA1c)
22,1000100164719,2016-09-12,7.1,Hemoglobin A1c (HbA1c)
68,1000100164439,2016-07-15,7.2,Hemoglobin A1c (HbA1c)
89,1000100164719,2014-06-02,7.2,Hemoglobin A1c (HbA1c)
90,1000100165244,2014-05-01,8.8,Hemoglobin A1c (HbA1c)
91,1000100164719,2014-03-18,7.8,Hemoglobin A1c (HbA1c)
139,1000100164439,2019-06-28,7.7,Hemoglobin A1c (HbA1c)
140,1000100415873,2019-04-07,7.5,Hemoglobin A1c (HbA1c)
141,1000100164439,2019-04-05,7.6,Hemoglobin A1c (HbA1c)


In [11]:
lo = pd.to_timedelta('7 days')
hi = pd.to_timedelta('14 days')
print(lo, hi)

7 days 00:00:00 14 days 00:00:00


# TimeDelta column

In [13]:
import datetime
import datetime as dt
df = alb_data
df['datetime'] = alb_data['PerformedDate'].apply(lambda x: 
                                    dt.datetime.strptime(x,'%Y-%m-%d'))
df = df.sort_values(by='datetime', ascending=True)
grouped = df.groupby("Patient_ID")
for name, group in grouped: 
    group['delta'] = (group['datetime']-group['datetime'].shift())
    group.drop(pd.Index(np.where((group['delta']<= lo) & (group['delta'] >=  hi))[0]), inplace = True)

In [14]:
grouped_lists = grouped['TestResult_calc'].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['TestResult_calc'] = grouped_lists['TestResult_calc'].apply(lambda x: list(set(x)))
data = grouped_lists[['TestResult_calc']]
print(data)

                                          TestResult_calc
0                               [8.2, 8.6, 8.4, 8.7, 8.1]
1                          [6.6, 6.0, 8.2, 9.1, 6.3, 8.6]
2       [7.5, 5.7, 6.3, 6.0, 6.1, 6.4, 6.5, 6.2, 6.6, ...
3                               [6.9, 7.3, 8.2, 6.8, 7.0]
4                          [5.8, 5.6, 5.9, 6.3, 6.0, 5.7]
...                                                   ...
128516                                         [7.1, 7.0]
128517                                         [8.7, 9.1]
128518                                    [7.2, 7.4, 7.8]
128519                                         [6.7, 6.8]
128520                                         [7.5, 7.0]

[128521 rows x 1 columns]


In [15]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
a = [0,0,0]
b = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
for index, row in data.iterrows() :
    #raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['TestResult_calc'].at[index]) > 15 :
        seq = data['TestResult_calc'].at[index]
        # choose a number of time steps
        n_steps = 3
        # split into samples
        X, y = split_sequence(seq, n_steps)
        y = y.reshape((-1, 1))
        a = np.vstack((a, X))
        b = np.vstack((b, y))
    else:
        continue
X = np.delete(a, (0), axis=0)
y = b[10:, :]
print(X)
print(y)

[[7.6 7.2 5.7]
 [7.2 5.7 6.5]
 [5.7 6.5 6. ]
 ...
 [8.8 7.3 7.6]
 [7.3 7.6 8.1]
 [7.6 8.1 8. ]]
[[6.5]
 [6. ]
 [6.2]
 ...
 [8.1]
 [8. ]
 [8.6]]


In [16]:
 import numpy as np
>>> from sklearn.model_selection import train_test_split

In [17]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, y_train, epochs=200, verbose=0)
result = model.evaluate(X_test,y_test)
print("test loss, test acc:", result)

866/866 [==============================] - 3s 3ms/step - loss: 0.5533
test loss, test acc: 0.5532848238945007


In [6]:
data = pd.read_csv('FinalNew.csv')
Albumin_Creatinine_data = data[['Patient_ID', 'PerformedDate', 'Albumin/Creatinine_Res']]
Albumin_Creatinine_data = Albumin_Creatinine_data.dropna()
Albumin_Creatinine_data = Albumin_Creatinine_data[Albumin_Creatinine_data['Albumin/Creatinine_Res'] < 300]
Albumin_Creatinine_data = Albumin_Creatinine_data.sort_values(by='PerformedDate', ascending=True)
grouped_patients = Albumin_Creatinine_data.groupby("Patient_ID")
grouped_lists = grouped_patients["Albumin/Creatinine_Res"].apply(list)
grouped_lists = grouped_lists.reset_index()
grouped_lists['Albumin/Creatinine_Res'] = grouped_lists['Albumin/Creatinine_Res'].apply(lambda x: list(set(x)))
data = grouped_lists[['Albumin/Creatinine_Res']]
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
a = [0,0,0]
b = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
for index, row in data.iterrows() :
    #raw_seq = data['Albumin/Creatinine_Res'].at[index]
    if len(data['Albumin/Creatinine_Res'].at[index]) > 8 :
        seq = data['Albumin/Creatinine_Res'].at[index]
        # choose a number of time steps
        n_steps = 3
        # split into samples
        X, y = split_sequence(seq, n_steps)
        y = y.reshape((-1, 1))
        a = np.vstack((a, X))
        b = np.vstack((b, y))
    else:
        continue
X = np.delete(a, (0), axis=0)
y = b[10:, :]
print(y)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=0)
for index, row in data.iterrows() :
# demonstrate prediction
        x_input = np.asarray(seq[-4:-1])
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        y_actual = np.asarray(seq[-1:])
        rms = sqrt(mean_squared_error(y_actual, yhat))
        print(yhat)
        print(y_actual)
        print(rms)

[[68.]
 [ 8.]
 [12.]
 ...
 [71.]
 [47.]
 [84.]]


KeyboardInterrupt: 